## import libraries

In [12]:
import pandas as pd
import geopandas as gpd
import numpy as np

import requests
import folium

from bs4 import BeautifulSoup

In [2]:
ecocrop_variables = pd.read_csv('../data/agricultural/EcoCrop/OpenCLIM/Ecocrop_variable_lookup.csv')

In [9]:
ecocrop_clim = pd.read_csv('../data/agricultural/EcoCrop/OpenCLIM/EcoCrop_DB2.csv')

In [10]:
ecocrop_clim

,EcoPortCode,ScientificName,AUTH,FAMNAME,SYNO,COMNAME,LIFO,HABI,LISPA,PHYS,...,KTMPR,KTMP,PHOTO,CLIZ,ABITOL,ABISUS,INTRI,PROSY,GMIN,GMAX
0,289,Abelmoschus esculentus,(L.) Moench,Magnoliopsida:Dilleniidae:Malvales:Malvaceae,Hibiscus esculentus L.,"abelmoskus, america-neri, bakhua mun, bamia, b...",herb,erect,annual,single stem,...,NaN,NaN,short day (<12 hours),"tropical wet & dry (Aw), tropical wet (Ar), st...",NaN,NaN,NaN,NaN,50.0,180.0
1,290,Abelmoschus manihot,(L.) Medic.,Magnoliopsida:Dilleniidae:Malvales:Malvaceae,"Hibiscus manihot L. (1753), Abelmoschus maniho...","neka (Simbo), bele (Fiji), pele (Tonga, Tuvalu...",shrub,erect,"annual, perennial",NaN,...,NaN,NaN,long day (>14 hours),"tropical wet & dry (Aw), tropical wet (Ar)",NaN,NaN,NaN,NaN,365.0,365.0
2,291,Abelmoschus moschatus,Medic.,Magnoliopsida:Dilleniidae:Malvales:Malvaceae,Hibiscus abelmoschus L. (1753).,"abelmosk, musk mallow, mushkdan, muskdana, kas...","herb, sub-shrub",prostrate/procumbent/semi-erect,"annual, biennial, perennial","deciduous, multi stem",...,NaN,NaN,NaN,tropical wet & dry (Aw),NaN,NaN,can become a weed,NaN,0.0,0.0
3,295,Acacia auriculiformis,Cunn. ex Benth.,Magnoliopsida:Rosidae:Fabales:Leguminosae,Racosperma auriculiforme (A. Cunn. ex Benth.) ...,"Papuan wattle, auri, ear leaf acacia, tuhkehn ...",tree,erect,perennial,single stem,...,NaN,NaN,short day (<12 hours),"tropical wet & dry (Aw), tropical wet (Ar)","fire, drought",NaN,NaN,NaN,180.0,240.0
4,297,Acacia farnesiana,(L.) Willd.,Magnoliopsida:Rosidae:Fabales:Leguminosae,"Mimosa farnesiana L., Mimosa acicularis Poir.,...","Ellington curse, klu, cassie, espino blanco, a...",tree,erect,perennial,single stem,...,NaN,NaN,short day (<12 hours),"tropical wet & dry (Aw), steppe or semiarid (B...",NaN,NaN,can become a weed,NaN,60.0,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,400000,Chamaecrista rotundifolia,Persoon,Leguminosae,Cassia rotundifolia,Pasto rastiero,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2564,400001,Acacia polyacantha,Willd.,Leguminosae,Acacia catechu,Black catechu,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2565,400002,Prosopis affinis,(Sw.) DC.,Leguminosae,Prosopis algarrobilla,Algarobilla,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2566,400003,Vicia dasycarpa,Roth.,Leguminosae,Vicia villosa ssp. varia,Hairy vetch,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [11]:
ecocrop_variables

,Variable,Explanation
0,AUTH,Authority
1,FAMNAME,Family
2,SYNO,Synonyms
3,COMNAME,Common names
4,LIFO,Life form
5,HABI,Habitat
6,LISPA,Life span
7,PHYS,Physiology
8,CAT,Category
9,PLAT,Plant attributes


In [146]:
#### UNCOMMENT TO SCRAPE ####
##
# # scrape USDA Hariness Zone from Plants for a Future

# # # Sample dataframe
# # data = {'ScientificName': ['Brassica oleracea var. gemmifera', 
# #                         'Solanum lycopersicum var. cerasiforme', 
# #                         'Zea mays']}

# # # Sample dataframe
# # data = {'ScientificName': ['Brassica oleracea var. gemmifera', 
# #                         'Solanum lycopersicum', 
# #                         'Zea mays var. saccharata']}

# # df = pd.DataFrame(data)

# # df = ecocrop_clim.iloc[:100].copy()
# df = ecocrop_clim.copy()

# # Function to extract plant parts
# def extract_parts(plant_name):
#     parts = plant_name.split(' var. ')
#     genus_species = parts[0].split(' ')
#     if len(parts) > 1:
#         return genus_species[0], genus_species[1], parts[1]
#     else:
#         return genus_species[0], genus_species[1], None
    
# # Function to generate URL
# def generate_url(row):
#     genus = row['Genus']
#     species = row['Species']
#     variety = row['Variety']
#     if variety:
#         return f"https://pfaf.org/user/Plant.aspx?LatinName={genus}+{species}+{variety}"
#     else:
#         return f"https://pfaf.org/user/Plant.aspx?LatinName={genus}+{species}"

# # Function to scrape USDA hardiness zone
# def scrape_usda_hardiness(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
#         usda_hardiness = soup.find('span', id='ContentPlaceHolder1_lblUSDAhardiness')
#         if usda_hardiness:
#             return usda_hardiness.text.strip()
#         return None
#     except requests.RequestException as e:
#         print(f"Error fetching {url}: {e}")
#         return None
    
# # Apply the extraction function
# df[['Genus', 'Species', 'Variety']] = df['ScientificName'].apply(lambda x: pd.Series(extract_parts(x)))

# # Apply the functions to the dataframe
# df['PFAF_URL'] = df.apply(generate_url, axis=1)
# df['USDA_HARDINESS_ZONE'] = df['PFAF_URL'].apply(scrape_usda_hardiness)

# # Drop the original columns if not needed
# # df.drop(columns=['ScientificName', 'PFAF_URL'], inplace=True)

# # Display the dataframe
# # print(df)
# df

In [68]:
df = pd.read_csv('../data/agricultural/EcoCrop/OpenCLIM/EcoCrop_DB2_usda_hardiness.csv')

In [69]:
len(list(df['USDA_HARDINESS_ZONE']))

2568

In [86]:
df[['ScientificName','USDA_HARDINESS_ZONE']]

,ScientificName,USDA_HARDINESS_ZONE
0,Abelmoschus esculentus,5-11
1,Abelmoschus manihot,8-11
2,Abelmoschus moschatus,8-11
3,Acacia auriculiformis,10-12
4,Acacia farnesiana,9-11
...,...,...
2563,Chamaecrista rotundifolia,9-11
2564,Acacia polyacantha,NaN
2565,Prosopis affinis,NaN
2566,Vicia dasycarpa,NaN


In [90]:
df.loc[2566, 'USDA_HARDINESS_ZONE']

nan

In [71]:
list(df['USDA_HARDINESS_ZONE'])
[(name, hardiness) for name, hardiness in zip(df['ScientificName'], df['USDA_HARDINESS_ZONE']) if hardiness==None]

[]

In [106]:
len([(name, hardiness) for name, hardiness in zip(df['ScientificName'], df['USDA_HARDINESS_ZONE']) if str(hardiness)=='nan'])

1371

In [100]:
df[df['Genus']=='Baikiaea']

,Unnamed: 0,EcoPortCode,ScientificName,AUTH,FAMNAME,SYNO,COMNAME,LIFO,HABI,LISPA,...,PROSY,GMIN,GMAX,Genus,Species,Variety,PFAF_URL,USDA_HARDINESS_ZONE,USDA_HARDINESS_ZONE_MIN,USDA_HARDINESS_ZONE_MAX
2259,2259,42707,Baikiaea plurijuga,Harms,Magnoliopsida:Rosidae:Fabales:Leguminosae,NaN,"Mukushi, Mukusi, Rhodesian teak, Umgusi, Zambe...",tree,erect,perennial,...,NaN,365.0,365.0,Baikiaea,plurijuga,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Bai...,10-12,10,12


In [116]:
# Using .at to set a single value
df.at[2259, 'USDA_HARDINESS_ZONE'] = '10-12'

In [133]:
# Function to split USDA hardiness zone into min and max
def split_hardiness_zone(zone):
    if pd.isna(zone):
        return pd.Series([None, None])
    if '-' in zone:
        min_zone, max_zone = zone.split('-')
        return pd.Series([min_zone, max_zone])
    else:
        return pd.Series([zone, zone])
    
# Apply the function to split the USDA hardiness zone
df[['USDA_HARDINESS_ZONE_MIN', 'USDA_HARDINESS_ZONE_MAX']] = df['USDA_HARDINESS_ZONE'].apply(split_hardiness_zone)

In [134]:
df

,Unnamed: 0,EcoPortCode,ScientificName,AUTH,FAMNAME,SYNO,COMNAME,LIFO,HABI,LISPA,...,PROSY,GMIN,GMAX,Genus,Species,Variety,PFAF_URL,USDA_HARDINESS_ZONE,USDA_HARDINESS_ZONE_MIN,USDA_HARDINESS_ZONE_MAX
0,0,289,Abelmoschus esculentus,(L.) Moench,Magnoliopsida:Dilleniidae:Malvales:Malvaceae,Hibiscus esculentus L.,"abelmoskus, america-neri, bakhua mun, bamia, b...",herb,erect,annual,...,NaN,50.0,180.0,Abelmoschus,esculentus,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,5-11,5,11
1,1,290,Abelmoschus manihot,(L.) Medic.,Magnoliopsida:Dilleniidae:Malvales:Malvaceae,"Hibiscus manihot L. (1753), Abelmoschus maniho...","neka (Simbo), bele (Fiji), pele (Tonga, Tuvalu...",shrub,erect,"annual, perennial",...,NaN,365.0,365.0,Abelmoschus,manihot,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,8-11,8,11
2,2,291,Abelmoschus moschatus,Medic.,Magnoliopsida:Dilleniidae:Malvales:Malvaceae,Hibiscus abelmoschus L. (1753).,"abelmosk, musk mallow, mushkdan, muskdana, kas...","herb, sub-shrub",prostrate/procumbent/semi-erect,"annual, biennial, perennial",...,NaN,0.0,0.0,Abelmoschus,moschatus,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Abe...,8-11,8,11
3,3,295,Acacia auriculiformis,Cunn. ex Benth.,Magnoliopsida:Rosidae:Fabales:Leguminosae,Racosperma auriculiforme (A. Cunn. ex Benth.) ...,"Papuan wattle, auri, ear leaf acacia, tuhkehn ...",tree,erect,perennial,...,NaN,180.0,240.0,Acacia,auriculiformis,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,10-12,10,12
4,4,297,Acacia farnesiana,(L.) Willd.,Magnoliopsida:Rosidae:Fabales:Leguminosae,"Mimosa farnesiana L., Mimosa acicularis Poir.,...","Ellington curse, klu, cassie, espino blanco, a...",tree,erect,perennial,...,NaN,60.0,240.0,Acacia,farnesiana,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,9-11,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,2563,400000,Chamaecrista rotundifolia,Persoon,Leguminosae,Cassia rotundifolia,Pasto rastiero,NaN,NaN,NaN,...,NaN,0.0,0.0,Chamaecrista,rotundifolia,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Cha...,9-11,9,11
2564,2564,400001,Acacia polyacantha,Willd.,Leguminosae,Acacia catechu,Black catechu,NaN,NaN,NaN,...,NaN,0.0,0.0,Acacia,polyacantha,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Aca...,NaN,None,None
2565,2565,400002,Prosopis affinis,(Sw.) DC.,Leguminosae,Prosopis algarrobilla,Algarobilla,NaN,NaN,NaN,...,NaN,0.0,0.0,Prosopis,affinis,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Pro...,NaN,None,None
2566,2566,400003,Vicia dasycarpa,Roth.,Leguminosae,Vicia villosa ssp. varia,Hairy vetch,NaN,NaN,NaN,...,NaN,0.0,0.0,Vicia,dasycarpa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Vic...,NaN,None,None


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2568 entries, 0 to 2567
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2568 non-null   int64  
 1   EcoPortCode              2568 non-null   int64  
 2   ScientificName           2568 non-null   object 
 3   AUTH                     2356 non-null   object 
 4   FAMNAME                  2386 non-null   object 
 5   SYNO                     757 non-null    object 
 6   COMNAME                  1655 non-null   object 
 7   LIFO                     2305 non-null   object 
 8   HABI                     2271 non-null   object 
 9   LISPA                    2285 non-null   object 
 10  PHYS                     1894 non-null   object 
 11  CAT                      2295 non-null   object 
 12  PLAT                     1289 non-null   object 
 13  TOPMN                    2071 non-null   float64
 14  TOPMX                   

In [141]:
df[df['USDA_HARDINESS_ZONE_MIN'] == df['USDA_HARDINESS_ZONE_MAX']]

,Unnamed: 0,EcoPortCode,ScientificName,AUTH,FAMNAME,SYNO,COMNAME,LIFO,HABI,LISPA,...,PROSY,GMIN,GMAX,Genus,Species,Variety,PFAF_URL,USDA_HARDINESS_ZONE,USDA_HARDINESS_ZONE_MIN,USDA_HARDINESS_ZONE_MAX
35,35,432,Apium graveolens var. dulce,L.,Magnoliopsida:Rosidae:Apiales:Umbelliferae,NaN,celery,herb,"erect, acaulescent (or rosette plants)","annual, biennial",...,"home garden, small scale (manual)",80.0,160.0,Apium,graveolens,dulce,https://pfaf.org/user/Plant.aspx?LatinName=Api...,Coming soon,Coming soon,Coming soon
42,42,481,Avena sativa,L.,Liliopsida:Commelinidae:Cyperales:Gramineae,NaN,"oat, oats, hawer, hafer, avena, avoine, avoine...",grass,erect,annual,...,NaN,110.0,270.0,Avena,sativa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Ave...,Coming soon,Coming soon,Coming soon
53,53,517,Bidens pilosa,Linnaeus,Magnoliopsida:Asteridae:Asterales:Compositae,"Bidens alba (L.) DC., Bidens pilosus var. albu...","beggar's tick, cobbler's pegs, common blackjac...",herb,erect,"annual, perennial",...,home garden,0.0,0.0,Bidens,pilosa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Bid...,Coming soon,Coming soon,Coming soon
70,70,555,Brassica oleracea var. gemmifera,L. [Zenker],Magnoliopsida:Dilleniidae:Capparales:Cruciferae,NaN,"Brussels sprouts, chou de Bruxelles (French), ...",herb,erect,"annual, biennial",...,"home garden, small scale (manual)",100.0,130.0,Brassica,oleracea,gemmifera,https://pfaf.org/user/Plant.aspx?LatinName=Bra...,Coming soon,Coming soon,Coming soon
106,106,693,Cichorium endivia,L.,Magnoliopsida:Asteridae:Asterales:Compositae,NaN,endive,herb,acaulescent (or rosette plants),"annual, biennial",...,"home garden, small scale (manual)",70.0,85.0,Cichorium,endivia,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Cic...,Coming soon,Coming soon,Coming soon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,2375,198033,Amelanchier pallida,Greene,Magnoliopsida:Rosidae:Rosales:Rosaceae,Amelanchier florida var. gracilis (Heller) M.E...,"pale serviceberry, pallid serviceberry, Utah s...",shrub,erect,perennial,...,NaN,0.0,0.0,Amelanchier,pallida,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Ame...,Coming soon,Coming soon,Coming soon
2401,2401,230348,Rhodiola rosea,L.,NaN,"Rhodiola minor Mill., 1768, Sedum Rosea (L.) S...","roseroot stonecrop, golden root, artic root, k...",herb,erect,perennial,...,home garden,0.0,0.0,Rhodiola,rosea,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Rho...,Coming soon,Coming soon,Coming soon
2409,2409,238008,Vicia villosa sp. varia,NaN,NaN,NaN,NaN,herb,climber/scrambler/scadent,"annual, biennial",...,large scale/commercial,0.0,0.0,Vicia,villosa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Vic...,Coming soon,Coming soon,Coming soon
2410,2410,238009,Vicia villosa ssp. villosa,Roth,Magnoliopsida:Rosidae:Fabales:Leguminosae,NaN,winter vetch,herb,"prostrate/procumbent/semi-erect, climber/scram...",annual,...,NaN,135.0,255.0,Vicia,villosa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Vic...,Coming soon,Coming soon,Coming soon


In [142]:
# Identify rows with zones of the form '8a', '10b', etc.
pattern = r'^\d+[a-zA-Z]$'
single_letter_zones = df[df['USDA_HARDINESS_ZONE'].str.match(pattern, na=False)]
single_letter_zones

,Unnamed: 0,EcoPortCode,ScientificName,AUTH,FAMNAME,SYNO,COMNAME,LIFO,HABI,LISPA,...,PROSY,GMIN,GMAX,Genus,Species,Variety,PFAF_URL,USDA_HARDINESS_ZONE,USDA_HARDINESS_ZONE_MIN,USDA_HARDINESS_ZONE_MAX


In [143]:
df[df['USDA_HARDINESS_ZONE'] == 'Coming soon']

,Unnamed: 0,EcoPortCode,ScientificName,AUTH,FAMNAME,SYNO,COMNAME,LIFO,HABI,LISPA,...,PROSY,GMIN,GMAX,Genus,Species,Variety,PFAF_URL,USDA_HARDINESS_ZONE,USDA_HARDINESS_ZONE_MIN,USDA_HARDINESS_ZONE_MAX
35,35,432,Apium graveolens var. dulce,L.,Magnoliopsida:Rosidae:Apiales:Umbelliferae,NaN,celery,herb,"erect, acaulescent (or rosette plants)","annual, biennial",...,"home garden, small scale (manual)",80.0,160.0,Apium,graveolens,dulce,https://pfaf.org/user/Plant.aspx?LatinName=Api...,Coming soon,Coming soon,Coming soon
42,42,481,Avena sativa,L.,Liliopsida:Commelinidae:Cyperales:Gramineae,NaN,"oat, oats, hawer, hafer, avena, avoine, avoine...",grass,erect,annual,...,NaN,110.0,270.0,Avena,sativa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Ave...,Coming soon,Coming soon,Coming soon
53,53,517,Bidens pilosa,Linnaeus,Magnoliopsida:Asteridae:Asterales:Compositae,"Bidens alba (L.) DC., Bidens pilosus var. albu...","beggar's tick, cobbler's pegs, common blackjac...",herb,erect,"annual, perennial",...,home garden,0.0,0.0,Bidens,pilosa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Bid...,Coming soon,Coming soon,Coming soon
70,70,555,Brassica oleracea var. gemmifera,L. [Zenker],Magnoliopsida:Dilleniidae:Capparales:Cruciferae,NaN,"Brussels sprouts, chou de Bruxelles (French), ...",herb,erect,"annual, biennial",...,"home garden, small scale (manual)",100.0,130.0,Brassica,oleracea,gemmifera,https://pfaf.org/user/Plant.aspx?LatinName=Bra...,Coming soon,Coming soon,Coming soon
106,106,693,Cichorium endivia,L.,Magnoliopsida:Asteridae:Asterales:Compositae,NaN,endive,herb,acaulescent (or rosette plants),"annual, biennial",...,"home garden, small scale (manual)",70.0,85.0,Cichorium,endivia,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Cic...,Coming soon,Coming soon,Coming soon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,2375,198033,Amelanchier pallida,Greene,Magnoliopsida:Rosidae:Rosales:Rosaceae,Amelanchier florida var. gracilis (Heller) M.E...,"pale serviceberry, pallid serviceberry, Utah s...",shrub,erect,perennial,...,NaN,0.0,0.0,Amelanchier,pallida,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Ame...,Coming soon,Coming soon,Coming soon
2401,2401,230348,Rhodiola rosea,L.,NaN,"Rhodiola minor Mill., 1768, Sedum Rosea (L.) S...","roseroot stonecrop, golden root, artic root, k...",herb,erect,perennial,...,home garden,0.0,0.0,Rhodiola,rosea,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Rho...,Coming soon,Coming soon,Coming soon
2409,2409,238008,Vicia villosa sp. varia,NaN,NaN,NaN,NaN,herb,climber/scrambler/scadent,"annual, biennial",...,large scale/commercial,0.0,0.0,Vicia,villosa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Vic...,Coming soon,Coming soon,Coming soon
2410,2410,238009,Vicia villosa ssp. villosa,Roth,Magnoliopsida:Rosidae:Fabales:Leguminosae,NaN,winter vetch,herb,"prostrate/procumbent/semi-erect, climber/scram...",annual,...,NaN,135.0,255.0,Vicia,villosa,NaN,https://pfaf.org/user/Plant.aspx?LatinName=Vic...,Coming soon,Coming soon,Coming soon


In [144]:
# Convert the new columns to integers
df['USDA_HARDINESS_ZONE_MIN'] = pd.to_numeric(df['USDA_HARDINESS_ZONE_MIN'], errors='coerce').astype('Int64')
df['USDA_HARDINESS_ZONE_MAX'] = pd.to_numeric(df['USDA_HARDINESS_ZONE_MAX'], errors='coerce').astype('Int64')

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2568 entries, 0 to 2567
Data columns (total 63 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               2568 non-null   int64  
 1   EcoPortCode              2568 non-null   int64  
 2   ScientificName           2568 non-null   object 
 3   AUTH                     2356 non-null   object 
 4   FAMNAME                  2386 non-null   object 
 5   SYNO                     757 non-null    object 
 6   COMNAME                  1655 non-null   object 
 7   LIFO                     2305 non-null   object 
 8   HABI                     2271 non-null   object 
 9   LISPA                    2285 non-null   object 
 10  PHYS                     1894 non-null   object 
 11  CAT                      2295 non-null   object 
 12  PLAT                     1289 non-null   object 
 13  TOPMN                    2071 non-null   float64
 14  TOPMX                   

In [147]:
df.to_csv('../data/EcoCrop/EcoCrop_DB2_usda_hardiness.csv')

## end